# Система обработки резюме и анализа данных о вакансиях

**Архитектура**: Паттерн Chain of Responsibility для модульной предобработки данных

**Назначение**: Загрузка CSV, трансформация признаков, подготовка матриц признаков (X) и целевой переменной (y) в формате NumPy

## Инициализация окружения и импорты

In [ ]:
from __future__ import annotations

import argparse
import logging
import re
import sys
from abc import ABC, abstractmethod
from pathlib import Path
from typing import Optional, Sequence, Tuple, List, Dict, Any, Set

import numpy as np
import pandas as pd

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

## Ядро архитектуры: цепочка обработчиков

In [ ]:
class ProcessorStep(ABC):
    """Абстрактный обработчик в цепочке ответственности.
    
    Каждый обработчик имеет уникальную ответственность и может передать
    результат следующему обработчику в цепочке.
    """

    def __init__(self) -> None:
        self._successor: Optional[ProcessorStep] = None

    def set_next(self, handler: ProcessorStep) -> ProcessorStep:
        """Установить следующий обработчик в цепочке."""
        self._successor = handler
        return handler

    def handle(self, data: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
        """Обработать данные и передать результат дальше."""
        if data is None:
            logger.warning(f'{self.__class__.__name__}: получены None данные')
            return None

        logger.info(f'{self.__class__.__name__}: обработка {data.shape[0]} строк, {data.shape[1]} колонок')
        
        processed = self.process(data)
        
        if self._successor is not None:
            return self._successor.handle(processed)
        return processed

    @abstractmethod
    def process(self, data: pd.DataFrame) -> pd.DataFrame:
        """Реализовать логику обработки."""
        raise NotImplementedError

## Конкретные обработчики данных

In [ ]:
class DataSourceLoader(ProcessorStep):
    """Загружает CSV файл с автоматическим определением кодировки."""

    SUPPORTED_ENCODINGS: Sequence[str] = ('utf-8', 'cp1251', 'iso-8859-5')

    def __init__(self, filepath: Path | str, **csv_options) -> None:
        super().__init__()
        self.filepath = Path(filepath)
        self.csv_options = csv_options

    def process(self, _: Optional[pd.DataFrame]) -> pd.DataFrame:
        if not self.filepath.exists():
            raise FileNotFoundError(f'CSV не найден: {self.filepath}')

        for encoding in self.SUPPORTED_ENCODINGS:
            try:
                logger.debug(f'Попытка загрузить с кодировкой: {encoding}')
                df = pd.read_csv(
                    self.filepath,
                    sep=',',
                    quotechar='"',
                    engine='python',
                    encoding=encoding,
                    index_col=0,
                    **self.csv_options
                )
                df.columns = [str(c).strip() for c in df.columns]
                logger.info(f'✓ Загрузка успешна с {encoding}')
                return df
            except (UnicodeDecodeError, pd.errors.ParserError):
                continue

        raise RuntimeError(f'Не удалось загрузить CSV ни с одной из кодировок')

In [ ]:
class TextFieldSanitizer(ProcessorStep):
    """Очистка текстовых полей от нежелательных символов."""

    @staticmethod
    def _sanitize(value: Any) -> Any:
        if not isinstance(value, str):
            return value
        
        s = value.replace('\\ufeff', '').replace('\\xa0', ' ')
        s = re.sub(r'[\\t\\n\\r]+', ' ', s)
        s = ''.join(ch for ch in s if ch.isprintable() or ch == ' ')
        s = re.sub(r'\\s+', ' ', s)
        return s.strip()

    def process(self, data: pd.DataFrame) -> pd.DataFrame:
        result = data.copy()
        text_cols = result.select_dtypes(include=['object', 'string']).columns
        
        for col in text_cols:
            result[col] = result[col].apply(self._sanitize)
            
        return result

In [ ]:
class SalaryNormalizer(ProcessorStep):
    """Преобразование зарплаты в единую валюту (рубли)."""

    CURRENCY_RATES: Dict[str, float] = {
        'rub': 1.0, 'руб': 1.0, 'руб.': 1.0,
        'usd': 78.5, 'eur': 91.0,
        'kzt': 0.152, 'uah': 1.8,
        'gbp': 105.0, 'cny': 11.2,
    }

    def __init__(self, salary_column: str = 'ЗП') -> None:
        super().__init__()
        self.salary_column = salary_column

    def process(self, data: pd.DataFrame) -> pd.DataFrame:
        if self.salary_column not in data.columns:
            return data

        result = data.copy()
        numbers = result[self.salary_column].astype(str).str.extract(r'(\\d+[.,]?\\d*)', expand=False)
        numbers = pd.to_numeric(numbers.str.replace(',', '.'), errors='coerce')
        
        currencies = result[self.salary_column].astype(str).str.extract(r'([A-Za-zА-Яа-яёЁ.]+)', expand=False).fillna('')
        currencies = currencies.str.lower().str.rstrip('.')
        
        def convert_to_rub(amount: float, currency: str) -> Optional[float]:
            if pd.isna(amount):
                return None
            rate = self.CURRENCY_RATES.get(currency, 1.0)
            return amount * rate
        
        result[self.salary_column] = [convert_to_rub(a, c) for a, c in zip(numbers, currencies)]
        return result

In [ ]:
class OutlierProcessor(ProcessorStep):
    """Обработка выбросов в целевой переменной через IQR метод."""

    def __init__(self, target_col: str = 'ЗП', multiplier: float = 1.5, mode: str = 'clip') -> None:
        super().__init__()
        self.target_col = target_col
        self.multiplier = multiplier
        self.mode = mode

    def process(self, data: pd.DataFrame) -> pd.DataFrame:
        if self.target_col not in data.columns:
            return data

        result = data.copy()
        target = result[self.target_col].dropna()
        
        q1 = target.quantile(0.25)
        q3 = target.quantile(0.75)
        iqr_range = q3 - q1
        
        lower_bound = q1 - self.multiplier * iqr_range
        upper_bound = q3 + self.multiplier * iqr_range
        
        outlier_mask = (result[self.target_col] < lower_bound) | (result[self.target_col] > upper_bound)
        
        if self.mode == 'clip':
            result[self.target_col] = result[self.target_col].clip(lower=lower_bound, upper=upper_bound)
        elif self.mode == 'remove':
            result = result.loc[~outlier_mask].reset_index(drop=True)
        
        return result

In [ ]:
class DataQualityEnforcer(ProcessorStep):
    """Работа с полнотой и качеством данных."""

    def __init__(self, remove_dups: bool = True, nan_threshold: float = 0.5) -> None:
        super().__init__()
        self.remove_dups = remove_dups
        self.nan_threshold = nan_threshold

    def process(self, data: pd.DataFrame) -> pd.DataFrame:
        result = data.copy()
        
        if self.remove_dups and len(result) > 50:
            result = result.drop_duplicates()
        
        threshold = int(self.nan_threshold * len(result))
        result = result.dropna(axis=1, thresh=threshold)
        
        numeric_cols = result.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            result[col] = result[col].fillna(result[col].median())
        
        categorical_cols = result.select_dtypes(include=['object', 'category']).columns
        for col in categorical_cols:
            result[col] = result[col].fillna('__undefined__')
        
        return result

## Главный энкодер признаков

In [ ]:
class FeatureEngineer(ProcessorStep):
    """Комплексный энкодер признаков с множеством трансформаций."""

    POSITION_GROUPS: Dict[str, List[str]] = {
        'development': ['программист', 'разработчик', 'developer', 'java', 'python'],
        'infrastructure': ['системный', 'администратор', 'devops'],
        'management': ['менеджер', 'руководитель', 'project manager'],
        'analytics': ['аналитик', 'data', 'bi'],
        'customer_support': ['поддержка', 'support'],
        'other': []
    }

    MAJOR_CITIES: Set[str] = {'москва', 'мск', 'московская область'}
    SPB_CITIES: Set[str] = {'санкт-петербург', 'спб'}
    LARGE_CITIES: Set[str] = {'новосибирск', 'екатеринбург', 'казань', 'нижний новгород'}

    @staticmethod
    def _extract_gender(text: str) -> Optional[int]:
        if not isinstance(text, str):
            return None
        t = text.lower()
        if 'муж' in t or 'male' in t:
            return 1
        if 'жен' in t or 'female' in t:
            return 0
        return None

    @staticmethod
    def _extract_age(text: str) -> Optional[float]:
        if not isinstance(text, str):
            return None
        m = re.search(r'(\\d{1,3})\\s*(?:лет|год|г|years?)', text.lower())
        if m:
            age = float(m.group(1))
            return age if 15 <= age <= 80 else None
        return None

    @classmethod
    def _classify_position(cls, title: str) -> str:
        if not isinstance(title, str):
            return 'other'
        t = title.lower()
        for group, keywords in cls.POSITION_GROUPS.items():
            if any(kw in t for kw in keywords):
                return group
        return 'other'

    @classmethod
    def _classify_city(cls, city_name: str) -> str:
        if not isinstance(city_name, str):
            return 'small_city'
        c = city_name.lower().split(',')[0].strip()
        if any(k in c for k in cls.MAJOR_CITIES):
            return 'moscow_region'
        if any(k in c for k in cls.SPB_CITIES):
            return 'spb_region'
        if c in cls.LARGE_CITIES:
            return 'large_city'
        return 'small_city'

    @staticmethod
    def _extract_experience(text: str) -> Optional[float]:
        if not isinstance(text, str) or 'не указано' in text.lower():
            return None
        m = re.search(r'(\\d+)\\s*(?:лет|год|г|years?)', text.lower())
        if m:
            return float(m.group(1))
        return None

    def _transform_demographics(self, df: pd.DataFrame) -> pd.DataFrame:
        src_col = 'Пол, возраст'
        if src_col not in df.columns:
            return df
        out = df.copy()
        out['sex'] = out[src_col].apply(self._extract_gender).fillna(-1)
        out['age_years'] = out[src_col].apply(self._extract_age)
        out['age_years'] = out['age_years'].fillna(out['age_years'].median()).clip(18, 75)
        out.drop(columns=[src_col], inplace=True, errors='ignore')
        return out

    def _transform_position_target(self, df: pd.DataFrame) -> pd.DataFrame:
        src_col = 'Ищет работу на должность:'
        if src_col not in df.columns:
            return df
        out = df.copy()
        out['position_group'] = out[src_col].apply(self._classify_position)
        out = pd.get_dummies(out, columns=['position_group'], prefix='pos', drop_first=True)
        out.drop(columns=[src_col], inplace=True, errors='ignore')
        return out

    def _transform_location(self, df: pd.DataFrame) -> pd.DataFrame:
        src_col = 'Город'
        if src_col not in df.columns:
            return df
        out = df.copy()
        out['city_cluster'] = out[src_col].apply(self._classify_city)
        out = pd.get_dummies(out, columns=['city_cluster'], drop_first=True)
        out.drop(columns=[src_col], inplace=True, errors='ignore')
        return out

    def _transform_employment(self, df: pd.DataFrame) -> pd.DataFrame:
        src_col = 'Занятость'
        if src_col not in df.columns:
            return df
        out = df.copy()
        out['full_time'] = out[src_col].apply(lambda x: 1 if isinstance(x, str) and 'полная' in x.lower() else 0)
        out['part_time'] = out[src_col].apply(lambda x: 1 if isinstance(x, str) and 'частична' in x.lower() else 0)
        out.drop(columns=[src_col], inplace=True, errors='ignore')
        return out

    def _transform_experience(self, df: pd.DataFrame) -> pd.DataFrame:
        src_col = 'Опыт (двойное нажатие для полной версии)'
        if src_col not in df.columns:
            return df
        out = df.copy()
        out['work_years'] = out[src_col].apply(self._extract_experience)
        out['work_years'] = out['work_years'].fillna(out['work_years'].median()).clip(0, 50)
        out.drop(columns=[src_col], inplace=True, errors='ignore')
        return out

    def process(self, data: pd.DataFrame) -> pd.DataFrame:
        df = data.copy()
        df = self._transform_demographics(df)
        df = self._transform_position_target(df)
        df = self._transform_location(df)
        df = self._transform_employment(df)
        df = self._transform_experience(df)
        return df

## Экспорт и сохранение результатов

In [ ]:
class DatasetSplitter(ProcessorStep):
    """Разделение DataFrame на признаки (X) и целевую переменную (y)."""

    KNOWN_TARGETS: List[str] = ['ЗП', 'Зарплата', 'salary', 'target', 'y']

    def __init__(self, target_name: Optional[str] = None) -> None:
        super().__init__()
        self.target_name = target_name

    def process(self, data: pd.DataFrame) -> pd.DataFrame:
        return data

    def split(self, data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
        target_col = self._find_target_column(data)
        X = data.drop(columns=[target_col])
        y = data[target_col]
        logger.info(f'Разделение: X shape={X.shape}, y shape={y.shape}')
        return X, y

    def _find_target_column(self, data: pd.DataFrame) -> str:
        if self.target_name is not None:
            return self.target_name
        for name in self.KNOWN_TARGETS:
            if name in data.columns:
                logger.info(f'Целевая колонка: {name}')
                return name
        return data.columns[-1]

In [ ]:
class NumpyPersister(ProcessorStep):
    """Сохраняет обработанные данные в формате NumPy."""

    def __init__(self, source_path: Path | str, features_file: str = 'X.npy', target_file: str = 'y.npy', target_name: Optional[str] = None) -> None:
        super().__init__()
        self.output_dir = Path(source_path).parent
        self.features_file = features_file
        self.target_file = target_file
        self.splitter = DatasetSplitter(target_name)

    def process(self, data: pd.DataFrame) -> pd.DataFrame:
        X, y = self.splitter.split(data)
        
        X_path = self.output_dir / self.features_file
        y_path = self.output_dir / self.target_file
        
        X_array = X.astype(float).to_numpy()
        y_array = y.astype(float).to_numpy()
        
        np.save(X_path, X_array)
        np.save(y_path, y_array)
        
        logger.info(f'✓ Сохранено: {X_path} {X_array.shape}')
        logger.info(f'✓ Сохранено: {y_path} {y_array.shape}')
        
        return data

## Оркестрация пайплайна

In [ ]:
def assemble_pipeline(csv_path: Path, target_col: Optional[str] = None) -> ProcessorStep:
    """Собрать полный пайплайн из отдельных обработчиков."""
    
    loader = DataSourceLoader(csv_path)
    sanitizer = TextFieldSanitizer()
    salary_norm = SalaryNormalizer(salary_column='ЗП')
    outlier_proc = OutlierProcessor(target_col='ЗП')
    quality_proc = DataQualityEnforcer()
    features = FeatureEngineer()
    persister = NumpyPersister(csv_path, target_name=target_col)
    
    loader.set_next(sanitizer).set_next(salary_norm).set_next(outlier_proc).set_next(quality_proc).set_next(features).set_next(persister)
    
    logger.info('Пайплайн собран')
    return loader


def execute_pipeline(csv_path: Path, target_col: Optional[str] = None) -> pd.DataFrame:
    """Выполнить обработку CSV."""
    loader = assemble_pipeline(csv_path, target_col)
    result = loader.handle(None)
    return result

## Точка входа

In [ ]:
def main(args: Optional[List[str]] = None) -> None:
    """Главная функция обработки данных.
    
    Использование: python app.py path/to/hh.csv
    """
    parser = argparse.ArgumentParser(description='Обработка данных вакансий')
    parser.add_argument('input_file', type=str, help='Путь к CSV файлу')
    parser.add_argument('--target', type=str, default=None, help='Целевая колонка')
    
    parsed = parser.parse_args(args)
    input_path = Path(parsed.input_file)
    
    if not input_path.exists():
        logger.error(f'Файл не найден: {input_path}')
        sys.exit(1)
    
    if input_path.suffix.lower() != '.csv':
        logger.error(f'Неподдерживаемый формат: {input_path.suffix}')
        sys.exit(1)
    
    logger.info(f'Начало обработки: {input_path}')
    
    try:
        df_processed = execute_pipeline(input_path, parsed.target)
        logger.info(f'✓ Обработка завершена! Форма: {df_processed.shape}')
    except Exception as e:
        logger.error(f'Ошибка: {e}', exc_info=True)
        sys.exit(1)


if __name__ == '__main__':
    main()

## Демонстрация

In [ ]:
input_file = Path('hh.csv')
target_column = 'ЗП'

if input_file.exists():
    logger.info(f'Начинаем обработку {input_file}')
    processed_data = execute_pipeline(input_file, target_column)
    logger.info('✓ Обработка завершена!')
    
    x_path = input_file.parent / 'X.npy'
    y_path = input_file.parent / 'y.npy'
    
    if x_path.exists() and y_path.exists():
        x_loaded = np.load(x_path)
        y_loaded = np.load(y_path)
        logger.info(f'X shape: {x_loaded.shape}')
        logger.info(f'y shape: {y_loaded.shape}')
else:
    logger.warning(f'Файл {input_file} не найден')